# `ETraceState`: 在线学习状态管理


在 `braintrace` 框架中，`ETraceState` 类系列提供了强大的状态管理功能，专门用于实现**资格迹（Eligibility Trace）在线学习机制**。资格迹是强化学习和神经网络训练中的重要概念，它允许系统追踪和更新神经元及突触的历史活动，从而实现更高效的学习算法。

## 核心特性

- **状态追踪**: 实时记录神经元和突触的动态状态
- **在线学习**: 支持基于资格迹的实时参数更新
- **灵活架构**: 适用于单个神经元、多房室模型和复杂树状结构
- **高效计算**: 基于 JAX 的高性能数值计算


In [21]:
import brainstate
import brainunit as u
import jax.numpy as jnp
import brainpy
import braintrace


## `brainstate.HiddenState` 类：单一状态管理

[`brainstate.HiddenState`](../apis/generated/brainstate.HiddenState.rst) 类实际上是`brainstate.HiddenState`的一个子类，，专门设计用于管理单个神经元或突触的状态变量。每个实例只能表示一个状态维度，确保了状态管理的清晰性和可控性。


### 实际应用：GIF 神经元模型

**广义积分发放（GIF）神经元**是一种先进的神经元模型，具有适应性机制和动态阈值。其数学描述如下：


$$
\begin{aligned}
\frac{\mathrm{d} I_1}{\mathrm{d} t} &= - k_1 I_1 \quad \text{(适应性电流1)} \\
\frac{\mathrm{d} I_2}{\mathrm{d} t} &= - k_2 I_2 \quad \text{(适应性电流2)} \\
\tau \frac{\mathrm{d} V}{\mathrm{d} t} &= - (V - V_{\mathrm{rest}}) + R\sum_{j}I_j + RI \quad \text{(膜电位)} \\
\frac{\mathrm{d} V_{\mathrm{th}}}{\mathrm{d} t} &= a(V - V_{\mathrm{rest}}) - b(V_{\mathrm{th}} - V_{\mathrm{th}\infty}) \quad \text{(动态阈值)}
\end{aligned}
$$


当 $V > V_{\mathrm{th}}$ 时，神经元发放并执行重置：

$$
\begin{aligned}
I_1 &\leftarrow R_1 I_1 + A_1 \\
I_2 &\leftarrow R_2 I_2 + A_2 \\
V &\leftarrow V_{\mathrm{reset}} \\
V_{\mathrm{th}} &\leftarrow \max(V_{\mathrm{th,reset}}, V_{\mathrm{th}})
\end{aligned}
$$

其中，$V$ 是膜电位，$V_{rest}$ 是静息电位，$R$ 是膜电阻，$I$ 是输入电流，$V_{th}$ 是阈值电位，$V_{th\infty}$ 是静息阈值电位，$a$ 和 $b$ 是阈值动力学参数，$I_j$ 是适应性电流，表示任意数量的内部电流，$R_j$ 是适应性电流的衰减系数，$A_j$ 是适应性电流的增量，$V_\mathrm{reset}$ 是重置电位，$V_{th, \mathrm{reset}}$ 是阈值重置电位。


对于GIF神经元模型，我们需要使用四个`brainstate.HiddenState`类来定义它的四个状态。以下是一个示例代码：

In [22]:
class GIF(brainpy.state.Neuron):
    """广义积分发放神经元模型

    实现了具有双重适应性电流和动态阈值的神经元模型
    """

    def __init__(self, size, **kwargs):
        super().__init__(size, **kwargs)

        # 模型参数
        self.tau = 20.0 * u.ms  # 膜时间常数
        self.R = 100.0 * u.ohm  # 膜电阻
        self.V_rest = -70.0 * u.mV  # 静息电位
        self.V_reset = -80.0 * u.mV  # 重置电位
        self.V_th_inf = -50.0 * u.mV  # 静息阈值

        # 适应性参数
        self.k1 = 0.1 / u.ms  # 适应性电流1衰减率
        self.k2 = 0.05 / u.ms  # 适应性电流2衰减率
        self.R1, self.R2 = 0.9, 0.8  # 发放时的衰减系数
        self.A1 = 10.0 * u.nA  # 适应性电流1增量
        self.A2 = 5.0 * u.nA  # 适应性电流2增量

        # 阈值动力学参数
        self.a = 0.1 / u.ms  # 阈值适应强度
        self.b = 0.02 / u.ms  # 阈值恢复强度

    def init_state(self, *args, **kwargs):
        # 适应性电流（初始化为零）
        self.I1 = brainstate.HiddenState(jnp.zeros(self.varshape) * u.nA)
        self.I2 = brainstate.HiddenState(jnp.zeros(self.varshape) * u.nA)

        # 膜电位（随机初始化接近静息电位）
        self.V = brainstate.HiddenState(
            brainstate.random.normal(self.varshape) * 2.0 * u.mV + self.V_rest
        )

        # 动态阈值（随机初始化）
        self.V_th = brainstate.HiddenState(
            brainstate.random.uniform(-52.0, -48.0, self.varshape) * u.mV
        )


In [23]:
# 创建并初始化神经元群体
gif_neurons = GIF(size=100)  # 100个神经元
gif_neurons.init_state()

每个`brainstate.HiddenState`类实例都代表一个状态变量，分别对应于GIF神经元模型中的$I_1$、$I_2$、$V$和$V_{th}$。每个状态变量只能定义神经元或突触的一个状态。

In [24]:
print(f"每个状态变量管理的维度数: {gif_neurons.I1.num_state}")
print(f"神经元群体形状: {gif_neurons.I1.varshape}")

每个状态变量管理的维度数: 1
神经元群体形状: (100,)


### 关键特性说明

- **独立性**: 每个 `ETraceState` 实例管理一个独立的状态变量
- **类型安全**: 支持物理单位，确保计算的正确性
- **追踪能力**: 自动记录状态变化历史，支持资格迹计算


## `braintrace.ETraceGroupState` 类：群组状态管理

[`braintrace.ETraceGroupState`](../apis/generated/braintrace.ETraceGroupState.rst) 类专门用于定义神经元或突触群体的多个状态。它是`brainstate.HiddenState`类的一个子类，继承了其所有属性和方法。

在多房室神经元模型(multi-compartment neuron model)中，每一个变量表示多个房室的状态，比如膜电位。如果将每个房室的膜电位使用一个`brainstate.HiddenState`类来定义，那么在多房室神经元模型中就需要定义多个状态变量，这样会导致代码冗长且难以维护。然而，使用`braintrace.ETraceGroupState`类可以将多个状态变量组合在一起，简化代码结构。

在以下示例中，我们将使用`braintrace.ETraceGroupState`类来定义一个三房室神经元模型的状态变量。

### 多房室神经元建模

In [25]:
import braincell

# Instantiate a Morphology object
morphology = braincell.Morphology()

# Create individual sections using the creation methods
morphology.add_cylinder_section('soma', length=20 * u.um, diam=10 * u.um, nseg=1)  # Soma section
morphology.add_cylinder_section('axon', length=100 * u.um, diam=1 * u.um, nseg=2)  # Axon section
morphology.add_point_section(
    'dendrite',
    positions=[[0, 0, 0], [100, 0, 0], [200, 0, 0]] * u.um,
    diams=[2, 3, 2] * u.um,
    nseg=3
)  # Dendrite section with explicit points and diameters

# Connect the sections: axon and dendrite connected to soma
morphology.connect('axon', 'soma', parent_loc=1.0)  # Axon connects to soma at the end
morphology.connect('dendrite', 'soma', parent_loc=1.0)  # Dendrite connects to soma at the end

# Print a summary of the morphology
morphology

Morphology(
  sections={
    'soma': Section<name='soma', nseg=1, points=2, Ra=100.0 * ohm * cmetre, cm=1.0 * ufarad / cmeter2, parent=None, parent_loc=None>,
    'axon': Section<name='axon', nseg=2, points=2, Ra=100.0 * ohm * cmetre, cm=1.0 * ufarad / cmeter2, parent='soma', parent_loc=1.0>,
    'dendrite': Section<name='dendrite', nseg=3, points=3, Ra=100.0 * ohm * cmetre, cm=1.0 * ufarad / cmeter2, parent='soma', parent_loc=1.0>
  }
)

In [26]:
class ThreeCompartmentNeuron(braincell.MultiCompartment):
    def __init__(self, pop_size, morphology):
        super().__init__(pop_size, morphology=morphology)

    def init_state(self, *args, **kwargs):
        self.V = braintrace.ETraceGroupState(jnp.zeros(self.varshape) * u.mV)

In [27]:
multi_neuron = ThreeCompartmentNeuron(10, morphology)
multi_neuron.init_state()

每个`braintrace.ETraceGroupState`实例都代表一个状态变量，包含了多个房室的状态信息。在这个例子中，我们只定义了膜电位$V$的状态变量，但实际上可以定义更多的状态变量，比如适应性电流$I_j$等。每个状态变量可以包含多个房室的状态信息，这样就可以在多房室神经元模型中使用。

In [28]:
print(f"总房室数量: {multi_neuron.V.num_state}")
print(f"状态形状: {multi_neuron.V.varshape}")
print(f"神经元群体大小: {multi_neuron.pop_size}")

总房室数量: 6
神经元群体大小: (10,)
状态形状: (10,)


### 优势分析

- **统一管理**: 单个状态对象管理多个房室，简化代码结构
- **空间一致性**: 保持房室间的空间关系和计算一致性
- **高效计算**: 向量化操作提高计算效率


## `braintrace.ETraceTreeState` 类：树状结构状态


[`braintrace.ETraceTreeState`](../apis/generated/braintrace.ETraceTreeState.rst) 提供了最灵活的状态管理方案，支持 **PyTree 树状结构**，适用于具有复杂层次关系的神经网络模型。
它是`brainstate.HiddenState`类的一个子类，继承了其所有属性和方法。

以下以GIF模型为例，展示如何使用`braintrace.ETraceTreeState`类来定义一个树状结构的状态变量。

### 高级应用示例

In [29]:
class GIF_tree(brainpy.state.Neuron):
    def init_state(self, *args, **kwargs):
        self.state = braintrace.ETraceTreeState(
            {
                'I1': jnp.zeros(self.varshape) * u.mA,
                'I2': jnp.zeros(self.varshape) * u.mA,
                'V': brainstate.random.random(self.varshape) * u.mV,
                'Vth': brainstate.random.uniform(1, 2, self.varshape) * u.mV
            }
        )

In [30]:
gif_tree = GIF_tree(5)
gif_tree.init_state()

每个`braintrace.ETraceTreeState`实例都代表一个树状结构的状态变量，包含了多个子状态变量。在这个例子中，我们定义了$I_1$、$I_2$、$V$和$V_{th}$四个状态变量，它们被组织成一个树状结构。

In [31]:
print(f"状态树包含的独立状态数: {gif_tree.state.num_state}")

状态树包含的独立状态数: 4


### 树状结构的优势

- **层次组织**: 逻辑清晰地组织复杂的状态变量
- **灵活访问**: 支持嵌套访问和部分更新
- **扩展性强**: 易于添加新的状态维度和功能模块
- **类型多样**: 支持不同类型和形状的状态变量


## 总结

`braintrace` 的 `ETraceState` 类系列为神经网络建模提供了强大而灵活的状态管理解决方案：

| 类型 | 适用场景 | 优势 | 典型应用 |
|------|----------|------|----------|
| `ETraceState` | 单一状态变量 | 简单直观，类型安全 | 基础神经元模型 |
| `ETraceGroupState` | 同质多状态 | 统一管理，高效计算 | 多房室神经元 |
| `ETraceTreeState` | 复杂层次结构 | 灵活组织，易于扩展 | 高级神经网络模型 |

选择合适的状态管理类型，可以显著提高代码的可读性、维护性和计算效率，为构建复杂的神经网络模型奠定坚实基础。